# <img src="https://stage.sunsolve.com/_astro/SunSolveLogo.BnIkxJwG_ZSGGx1.svg">
# SunSolve P90

## Intro

This notebook provides a tool for simulating uncertainty in PV systems using the '**SunSolve P90**' tool.

It can be used to introduce an uncertainty model to any other yield model, such as one from [SunSolve Yield](https://sunsolve.info/yield/).

Each code cell contains code that the user should adjust to their situation, then run.

A code cell is run with `SHIFT+ENTER`, which also moves to the next cell. `CTRL/⌘+ENTER` runs the cell without jumping forward. Colab users can press the play button to the left of the cell.

See [docs.sunsolve.com/en/p90](https://docs.sunsolve.com/en/p90) for the full documentation of SunSolve P90.

## Install P90 client and choose project folder

<details>
<summary>Click to expand for detailed instructions</summary>

In **Code cell 1**, you will install and login to SunSolve P90, using your PV Lighthouse account.

To obtain an account just contact <support@sunsolve.com>. (**TODO**: add onboarding link instead of email)

Press `SHIFT+ENTER` to run the cell (or the play button on the left for Colab users).
</details>

## Code cell 1 (**skip this**)

In [9]:
# TODO: when PyPi package is released, this cell will be the login+install step. Currently can't login first, since the package must be manually installed to login.

In **Code cell 2**, you will install the P90 client to access SunSolve P90. Then you will login. (**TODO**: change to "you will load helper functions to prepare and plot your analysis", after PyPi release.)

**You can leave the settings as default**. The project folder will be assumed as the current directory if so. If it lacks the required files, they'll be cloned into a sub-folder 'SunSolve P90' which will become the project folder.

However, **if you are using Google Colab and want to connect it to your Google Drive** do the following:
1. Change `USING_DRIVE` to `True`.
2. Change `DRIVE_FOLDER` to the desired project folder on your drive. (Paths starting with "G:\\\\" are attempted to be converted, but you may need to manually adjust your mounted drive's structure in Colab).
3. Run the cell, connect to your Google account, press `Continue` twice on the pop-up.

You can use Colab without Drive, but will need to manually download your results. Be careful of Colab's 90 minute idle timeout for inactivity.

## Code cell 2

In [10]:
import os, sys
USING_DRIVE = False  # Set to True if using Google Colab and Drive. Set DRIVE_FOLDER below also.
DRIVE_FOLDER = r"/content/G/MyDrive/"    # Windows paths (G:\\ like) will be handled below, Mac paths (~/Google Drive/) also. Alternatively, write as "/content/G/MyDrive/..." or "/content/G/SharedDrives/...".
REQUIRED_FILES = ["Data", "README.md", "UncertaintyFunctions.ipynb",
    "sunsolve_p90_client-0.1.0.167+prerelease-py3-none-any.whl"]      # If these are found in current directory, then we assume we are in the right folder

# Set up Drive folder if using Colab + Drive (must enter DRIVE_FOLDER path above)
if USING_DRIVE and "google.colab" in sys.modules:
    %pip uninstall -y tensorflow google-ai-generativelanguage grpcio-status --quiet # Uninstall conflicting packages (Colab-specific)
    from google.colab import drive
    folder = DRIVE_FOLDER.replace("\\", "/").replace("G://", "/content/G/").replace("~/Google Drive/", "/content/G/")\
        .replace("/My Drive", "/MyDrive").replace("/Shared Drives", "/SharedDrives")    # Colab doesn't use spaces for these base folder names
    drive.mount("/content/G")   # Mount Google Drive to "/content/G" (A pop-up will ask for authorization, you must press continue twice but don't need to allow additional permissions)
    # Change the working directory to your project folder
    os.chdir(folder)
# Otherwise, we stay in the current directory.

# Check if the required files/folders are available, or clone them into a "SunSolve P90" sub-folder (new project folder)
if not all(i in os.listdir() for i in REQUIRED_FILES):
    if not "SunSolve P90" in os.listdir() or not all(i in os.listdir("SunSolve P90") for i in REQUIRED_FILES):
        os.makedirs("SunSolve P90", exist_ok=True)
        os.system("git clone --depth 1 https://github.com/SF-PVL/P90-Notebook.git 'SunSolve P90'")
    os.chdir("SunSolve P90")    # Make the new "SunSolve P90" folder the project folder

# Install the P90 client (Colab: the pip dependency error can be safely ignored)
%pip install sunsolve_p90_client-0.1.0.167+prerelease-py3-none-any.whl --quiet

# Load helper functions
%run "UncertaintyFunctions.ipynb"   # If this fails, try restarting the Python kernel and re-running this cell TODO: remove when PyPi package is released
print("Helper functions loaded.")

Note: you may need to restart the kernel to use updated packages.


2025-10-18 09:17:18,190 - INFO - Connected to P90 service at middleware.pvlighthouse.com.au


Connecting to PV Lighthouse...
Authenticating...
Authentication successful!
Loading time step data...


2025-10-18 09:17:36,549 - INFO - P90 client connection closed


Loaded 8760 weather data points
Helper functions loaded.


## Set up simulation inputs

**Code cell 3** below is where you will definine up your simulation inputs.

These define the expected conditions of your system, including its weather (from a `.pwv` or `.csv` file). In a later cell, we will introduce uncertainty to some of these.

There are many other inputs you can define, with only some set below. See [Simulation Inputs](https://docs.sunsolve.com/en/p90/simulation-inputs) in the P90 docs for more info.

## Code cell 3

In [11]:
# Set uncertainty simulation constants (time scales with N_SIMS*N_YEARS, 5000*10 ≈ 120 seconds)
N_SIMS = 1000
N_YEARS = 12
simulation_options = build_simulation_options(number_of_years=N_YEARS, number_of_simulations=N_SIMS)

# Collect weather data: sydney.pvw file is provided as part of the source
weather_file_path = "Data/sydney.pvw"
weather_data = load_weather_data(weather_file_path)     # Must be .pvw or a .csv (in the same format as 'Data/sydney.csv', use this as a base if making your own .csv weather file)

# Set up system
module_info = build_module_info(
    length=2.0, width=1.0, bifaciality=0.8)

system_info = build_system_info(
    modules_per_string=1,
    fallback_module_tilt_in_degrees=30,
    number_of_inverters=1,
    num_strings_per_inverter=1,
    row_pitch_in_m=5.6,
    azimuth_in_degrees=90            # Direction modules face for positive tilt. 0=North, 90=East, 180=South
)

electrical_settings = build_electrical_settings(
    inverter_efficiency=0.98,
    module_to_module_mismatch=0.01,
    string_wiring_loss=0.01,
    inverter_wiring_loss=0.01,
    max_power_tracking_loss=0.0
)

thermal_settings = build_thermal_settings(uc=25.0, uv=1.2, alpha=0.9)

optical_settings = build_optical_settings(fallback_albedo=0.2,
    fallback_soiling_front=0.02, fallback_soiling_rear=0.002)

operational_settings = build_operational_settings(annual_degradation_rate=0.005,
    curtailment=0.02, availability=0.98)

# Set up result options
p_values = [50, 90, 95]         # e.g. [50, 90, 95] -> P50, P90, P95
bin_min = 0.75                  # The minimum yield (normalised to the median: P50) to be binned on histograms
bin_delta = 0.01                # The bin size for histograms
result_options = build_result_options(p_min=bin_min, p_delta=bin_delta, p_values=p_values)

## Code cell 4 (optional)

**Code cell 4** is an (optional) interactive weather plotter, to check your weather file. Run it with `CTRL/⌘+ENTER`, then adjust the slider to view each day of your weather file.

(**Note**: if plots appear with duplicates, this is a VS Code issue. Try restarting your kernel and re-running code cells 1-4.)

In [ ]:
interactive_weather_plot(weather_file_path, weather_data)

## Introduce uncertainties

Now you will set up input distributions (probability density functions - PDFs) that will introduce uncertainty to specific inputs.

## Code cell 5 (optional)

**Code cell 5** below is an (optional) interactive PDF plotter, which can help you to visualise your distributions before you apply them.

If a distribution has a mean of 1, then it will have an equal probability of increasing or decreasing the input.

If it is skewed towards 0, then it is more likely to decrease the input.

See [Uncertainty Distributions/PDF Types](https://docs.sunsolve.com/en/p90/uncertainty-distributions/pdf-types/) in the P90 docs for more info.

In [ ]:
interactive_distribution_plot()

**Code cell 6** is where you will now create the distributions for specific inputs, putting them into a list to send with the P90 request.

**Target Inputs** are identified as 'DistributionInput.____', the possible inputs are:

GHI, DiffuseFraction, WindSpeed, Temperature, ModulePower, SpectralCorrection, SoilingFront, SoilingRear, Uc, Uv, Alpha, AnnualDegradationRate, Availability, YieldModifier, CircumsolarFraction, UndulatingGround, ExtraIrradiance, Curtailment, DCHealth, InverterToInverterMismatch, StringToStringMismatch, ModuleToModuleMismatch, CellToCellMismatch, InverterEfficiency, ModuleEfficiencyTemperatureCoefficient, Albedo, RearStructuralShadingFactor, RearTransmissionFactor

Each input can have a distibution applied on any of three levels:

**simToSim**: sim-to-sim variability, affecting all years for that simulation; **yearToYear**: year-to-year variability; **stepToStep**: hour-to-hour variability

Note: AnnualDegradationRate is a rate/year and should be positive, soiling should be positive.

See [Uncertainty Distributions/Syntax](https://docs.sunsolve.com/en/p90/uncertainty-distributions/syntax/) in the P90 docs for more info on the syntax, or just create your distributions in **Code cell 5** and then copy the syntax from the plot title.

## Code cell 6

In [12]:
distribution_list = [
    create_distribution(DistributionInput.AnnualDegradationRate, simToSim=["SkewedGaussian", 6, 1, 0.002], yearToYear=["Gaussian", 1, 0.02]),
    create_distribution(DistributionInput.GHI, simToSim=["Gaussian", 1, 0.06], yearToYear=["Gaussian", 1, 0.04], stepToStep=["SkewedGaussian", -2, 0.95, 0.05]),
    create_distribution(DistributionInput.Temperature, simToSim=["Gaussian", 1, 0.06], yearToYear=["Gaussian", 1, 0.04], stepToStep=["SkewedGaussian", -2, 0.95, 0.05]),
    create_distribution(DistributionInput.SoilingFront, simToSim=["Weibull", 0, 0.04, 2], yearToYear=["Gaussian", 0.08, 0.04]),
]

## Run SunSolve P90 analysis

**Code cell 7** is where the P90 request is put together and sent to SunSolve P90.

**summary** is a returned object containing the normalised yield distributions for each year of your simulation, along with P-values and yearly 'P50 deviations' (median yields normalised to the first year's median yield).

**used_inputs** contains the inputs used for the simulation. Many will have been set to default values if you didn't supply them.

See [Simulation Result](https://docs.sunsolve.com/en/p90/uncertainty-distributions/simulation-result/) in the P90 docs for more info.

## Code cell 7

In [13]:
# Build and send request
p90_request = build_request(
    time_step_data=weather_data,
    module=module_info,
    system=system_info,
    electrical=electrical_settings,
    optical=optical_settings,
    thermal=thermal_settings,
    operational=operational_settings,
    distributions=distribution_list,
    simulation_options=simulation_options,
    result_options=result_options
)

summary, used_inputs = request_analysis(p90_request)
print_year_one_p_values(summary)   # Print the requested P-values for year one

2025-10-18 09:18:01,206 - INFO - Connected to P90 service at middleware.pvlighthouse.com.au
2025-10-18 09:18:01,284 - INFO - Starting P90 analysis request


Starting uncertainty analysis...


2025-10-18 09:18:09,503 - INFO - === P90 Analysis Inputs Used ===
2025-10-18 09:18:09,505 - INFO - Module: Length=2.000m, Width=1.000m, HeightAboveGround=1.500m, PowerRatingAtSTC=460.0W, CellToCellMismatch=0.0040, EfficiencyTempCoeff=0.002950, Bifaciality=0.800
2025-10-18 09:18:09,510 - INFO - System: ModulesPerString=1, StringsPerInverter=1, NumberOfInverters=1
2025-10-18 09:18:09,515 - INFO - System Geometry: RowPitch=5.60m, ModuleAzimuth=90.0°, FallbackModuleTilt=30.0°
2025-10-18 09:18:09,518 - INFO - System Tracking: TrackingCalculation=0, TiltLimit=55.0°
2025-10-18 09:18:09,519 - INFO - Electrical Efficiencies: InverterEff=0.9800, ModuleToModuleMismatch=0.0100, StringWiringLoss=0.0100
2025-10-18 09:18:09,520 - INFO - Electrical Losses: MaxPowerTracking=0.0000, InverterWiring=0.0100, StringToStringMismatch=0.0000, InverterToInverterMismatch=0.0000
2025-10-18 09:18:09,521 - INFO - Optical Multipliers: BeamFront=1.000, BeamRear=1.000, IsotropicFront=1.010, IsotropicRear=1.000
2025-10

Progress: 100.00%

2025-10-18 09:18:38,810 - INFO - Received P90 analysis summary


...

2025-10-18 09:18:38,849 - INFO - P90 client connection closed


###P-values (Year 1)

**P50**:	1.0000

**P90**:	0.8915

**P95**:	0.8593

In [ ]:
# Prints the P-values from the uncertainty summary for year one
def print_year_one_p_values(summary):
    display(Markdown("### P-values (Year 1)"))
    year_one_p_vals = [pval for pval in summary.YearlyPValue if pval.Year == 1]
    for p_val in year_one_p_vals:
        value = p_val.P50Deviation
        display(Markdown(f"**P{p_val.P}**:\t{value:.4f}"))
print_year_one_p_values(summary)

### P-values (Year 1)

**P50**:	1.0000

**P90**:	0.8915

**P95**:	0.8593

## Results

**Code cell 8** contains our main result plots.

Results are expressed in terms of 'relative yield' (normalised yield), where a value of 1 is equal to the P50 yield (median yield) of year one.

A P90 might have a relative yield of '0.88' - this means that 90% of simulated yields were at least 88% of the P50 yield.

The P50 yields of each year are also compared, where the 'Relative P50 yield' is defined relative to the year 1 P50 yield. Hence, this plot visualises annual degradation.

Absolute yield estimates are not provided, but [SunSolve Yield](https://sunsolve.info/yield/) can be used to estimate this accurately. You can also supply an absolute yield estimate for year one below, which will then scale all of the relative yields to absolute yields with this year 1 P50 value as the basis. It is assumed to be in 'MWh' if so, but can be changed by supplying a 'yield_units' string to the plotting functions (e.g. `yield_units="GJ"`).

Note: The first and last bins of the histograms are larger than the others, they include all values to their left and right, respectively.


See [Simulation Result](https://docs.sunsolve.com/en/p90/uncertainty-distributions/simulation-result/) in the P90 docs for more info.

## Code cell 8

In [ ]:
year_one_yield = 1              # Change this to plot absolute yields, if you know what the first year yield should be in MWh
if N_YEARS > 1:
    plot_yearly_P50_Values(summary, year_one_yield=year_one_yield)  # Plots degradation over N_YEARS, don't plot for single year
plot_interactive_histogram(summary, bin_min=bin_min, bin_delta=bin_delta, year_one_yield=year_one_yield)

## Code cell 9
**Code cell 9** below will allow you to plot your requested P-values as relative yields (relative to the year 1 P50 yield) across `N_YEARS` years.

In [94]:
# plot_pvalues(summary, year_one_yield=year_one_yield)
yearly_p_df

,P50,P90,P95
Year,,,
1,1.0,0.891478,0.859340
2,1.0,0.893447,0.872333
3,1.0,0.895356,0.864272
4,1.0,0.890738,0.866868
5,1.0,0.892910,0.860155
6,1.0,0.892761,0.857772
7,1.0,0.893063,0.852402
8,1.0,0.895399,0.866111
9,1.0,0.898133,0.864564


## Code cell 10 (export to Excel)

**Code cell 10** allows you to export all your results into an Excel spreadsheet.

Feel free to adjust the filename to your needs.

In [ ]:
date_prefix = datetime.date.today().strftime("%y%m%d_")
filename = date_prefix + f"P90 summary_{N_YEARS}x{N_SIMS}"
export_to_excel(summary, filename, bin_min=bin_min, bin_delta=bin_delta, used_inputs=used_inputs, dist_list=distribution_list)

CREATE_NOTEBOOK_COPY = False  # Set to True to create a copy of the current notebook with results saved
if CREATE_NOTEBOOK_COPY:
    # Create a copy of the current notebook
    current_notebook = "P90 Analysis.ipynb"
    result_notebook = filename.replace("summary", "result") + ".ipynb"

    try:
        with open(current_notebook, 'r', encoding='utf-8') as src:
            notebook_content = src.read()
        
        with open(result_notebook, 'w', encoding='utf-8') as dst:
            dst.write(notebook_content)
        
        print(f"Notebook copy created: {result_notebook}")
    except FileNotFoundError:
        print(f"Could not find {current_notebook} to copy. You may need to manually save your notebook.")
    except Exception as e:
        print(f"Error creating notebook copy: {e}")

Notebook copy created: 251019_P90 result_12x1000.ipynb


## Further analysis

Continue your own analysis below

In [ ]:
# Example: Plot a box plot of all years' relative yield distributions
fig = px.box(yearly_hist_df)
fig.update_layout(
    title=f"Relative yield distribution box plot{' (all-years)' if N_YEARS > 1 else '(single year)'}",
    xaxis_title="Yield (relative to yearly P50)",
    yaxis_title="Frequency"
)
fig.update_xaxes(dtick=5)
fig.show()
yearly_p_df

P_value,P50,P90,P95
Year,,,
1,1.0,0.891478,0.859340
2,1.0,0.893447,0.872333
3,1.0,0.895356,0.864272
4,1.0,0.890738,0.866868
5,1.0,0.892910,0.860155
6,1.0,0.892761,0.857772
7,1.0,0.893063,0.852402
8,1.0,0.895399,0.866111
9,1.0,0.898133,0.864564
